In [1]:
import pandas as pd
import pfr
import string

# Scrape combine data

In [2]:
combine_url_template = "http://www.pro-football-reference.com/draft/{}-combine.htm"
combine_row_css_selector = "#combine > tbody > tr" 
combine_col_css_selector = "#combine > thead > tr > th.poptip"
combine_dfs = []
combine_errors = []
for i in range (2000, 2019):
    url = combine_url_template.format(i)
    try:
        df = pfr.get_table(url, combine_row_css_selector, 
                           combine_col_css_selector, 0)
        df["Year"] = i
        combine_dfs.append(df)
    except Exception as e:
        combine_errors.append([url, e])

In [3]:
len(combine_dfs)

19

In [4]:
len(combine_errors)

0

In [5]:
combine_errors

[]

In [6]:
combine_df = pd.concat(combine_dfs, ignore_index=True)

In [7]:
combine_df.head()

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,BenchReps,BroadJump,3Cone,Shuttle,Drafted (tm/rnd/yr),Year
0,"(John Abraham, AbraJo00)",OLB,South Carolina,,6-4,252,4.55,,,,,,New York Jets / 1st / 13th pick / 2000,2000
1,"(Shaun Alexander, AlexSh00)",RB,Alabama,College Stats,6-0,218,4.58,,,,,,Seattle Seahawks / 1st / 19th pick / 2000,2000
2,"(Darnell Alford, AlfoDa20)",OT,Boston College,,6-4,334,5.56,25.0,23,94,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000
3,"(Kyle Allamon, None)",TE,Texas Tech,,6-2,253,4.97,29.0,,104,7.29,4.49,,2000
4,"(Rashard Anderson, AndeRa21)",CB,Jackson State,,6-2,206,4.55,34.0,,123,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000


In [8]:
combine_df.tail()

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,BenchReps,BroadJump,3Cone,Shuttle,Drafted (tm/rnd/yr),Year
6213,"(Chris Worley, WorlCh00)",ILB,Ohio St.,College Stats,6-2,238,4.86,29.5,15,,,,,2018
6214,"(Isaiah Wynn, WynnIs00)",G,Georgia,College Stats,6-3,313,,,,,,,,2018
6215,"(Isaac Yiadom, YiadIs00)",CB,Boston College,College Stats,6-1,190,4.52,,8,120,,4.18,,2018
6216,"(Kenny Young, None)",ILB,UCLA,College Stats,6-1,236,4.60,36.0,23,117,7.38,4.48,,2018
6217,"(Trevon Young, YounTr00)",EDGE,Louisville,College Stats,6-4,258,4.78,33.0,25,114,6.99,4.40,,2018


In [9]:
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6218 entries, 0 to 6217
Data columns (total 14 columns):
Player                 6218 non-null object
Pos                    6218 non-null object
School                 6218 non-null object
College                6218 non-null object
Ht                     6218 non-null object
Wt                     6218 non-null object
40yd                   6218 non-null object
Vertical               6218 non-null object
BenchReps              6218 non-null object
BroadJump              6218 non-null object
3Cone                  6218 non-null object
Shuttle                6218 non-null object
Drafted (tm/rnd/yr)    6218 non-null object
Year                   6218 non-null int64
dtypes: int64(1), object(13)
memory usage: 680.2+ KB


In [10]:
combine_df[["Player", "Pfr_ID"]] = combine_df.Player.apply(pd.Series)

In [11]:
combine_df.head()

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,BenchReps,BroadJump,3Cone,Shuttle,Drafted (tm/rnd/yr),Year,Pfr_ID
0,John Abraham,OLB,South Carolina,,6-4,252,4.55,,,,,,New York Jets / 1st / 13th pick / 2000,2000,AbraJo00
1,Shaun Alexander,RB,Alabama,College Stats,6-0,218,4.58,,,,,,Seattle Seahawks / 1st / 19th pick / 2000,2000,AlexSh00
2,Darnell Alford,OT,Boston College,,6-4,334,5.56,25.0,23,94,8.48,4.98,Kansas City Chiefs / 6th / 188th pick / 2000,2000,AlfoDa20
3,Kyle Allamon,TE,Texas Tech,,6-2,253,4.97,29.0,,104,7.29,4.49,,2000,None
4,Rashard Anderson,CB,Jackson State,,6-2,206,4.55,34.0,,123,7.18,4.15,Carolina Panthers / 1st / 23rd pick / 2000,2000,AndeRa21


In [12]:
combine_df.to_csv('data/combine_data_RAW_2018-03-13.csv', index=False)

# Scrape 3yr AV

In [13]:
# note year_min=2000 and year_max=2017
# will iterate through different values for offset to scrape
# all the pages of AV values
av_url_template = "http://www.pro-football-reference.com/play-index/psl_finder."\
                  "cgi?request=1&match=combined&year_min=2000&year_max=2017&seas"\
                  "on_start=1&season_end=3&pos%5B%5D=qb&pos%5B%5D=rb&pos%5B%5D=w"\
                  "r&pos%5B%5D=te&pos%5B%5D=e&pos%5B%5D=t&pos%5B%5D=g&pos%5B%5D="\
                  "c&pos%5B%5D=ol&pos%5B%5D=dt&pos%5B%5D=de&pos%5B%5D=dl&pos%5B%"\
                  "5D=ilb&pos%5B%5D=olb&pos%5B%5D=lb&pos%5B%5D=cb&pos%5B%5D=s&po"\
                  "s%5B%5D=db&pos%5B%5D=k&pos%5B%5D=p&draft_year_min=1936&draft_"\
                  "year_max=2017&draft_slot_min=1&draft_slot_max=500&draft_pick_"\
                  "in_round=pick_overall&conference=any&draft_pos%5B%5D=qb&draft"\
                  "_pos%5B%5D=rb&draft_pos%5B%5D=wr&draft_pos%5B%5D=te&draft_pos"\
                  "%5B%5D=e&draft_pos%5B%5D=t&draft_pos%5B%5D=g&draft_pos%5B%5D="\
                  "c&draft_pos%5B%5D=ol&draft_pos%5B%5D=dt&draft_pos%5B%5D=de&dr"\
                  "aft_pos%5B%5D=dl&draft_pos%5B%5D=ilb&draft_pos%5B%5D=olb&draf"\
                  "t_pos%5B%5D=lb&draft_pos%5B%5D=cb&draft_pos%5B%5D=s&draft_pos"\
                  "%5B%5D=db&draft_pos%5B%5D=k&draft_pos%5B%5D=p&c5val=1.0&order"\
                  "_by=av&offset={}"

In [14]:
av_row_css_selector = "#results > tbody > tr" 
av_col_css_selector = "#results > thead > tr:nth-child(2) > th"
av_dfs = []
av_errors = []
# this range is found by manually checking out what
# the offset value is for the last page of results
# in 
for i in range (0, 7801, 100):  
    url = av_url_template.format(i)
    try:
        df = pfr.get_table(url, av_row_css_selector, 
                           av_col_css_selector, 1)
        av_dfs.append(df)
    except Exception as e:
        av_errors.append([url, e])

In [15]:
len(av_dfs)

79

In [16]:
len(av_errors)

0

In [17]:
av_df = pd.concat(av_dfs, ignore_index=True)
av_df.head()

,Rk,Player,From,To,Draft,Tm,Lg,G,GS,Yrs,PB,AP1,AV
0,1,"(Cam Newton, NewtCa00)",2011,2013,1-1,CAR,NFL,48,48,3,2,0,51
1,2,"(Russell Wilson, WilsRu00)",2012,2014,3-75,SEA,NFL,48,48,3,2,0,51
2,3,"(Patrick Willis, WillPa98)",2007,2009,1-11,SFO,NFL,48,48,3,3,2,49
3,4,"(Patrick Peterson, PetePa00)",2011,2013,1-5,CRD,NFL,48,48,3,3,2,48
4,5,"(Aaron Donald, DonaAa00)",2014,2016,1-13,RAM,NFL,48,44,3,3,2,46


In [18]:
av_df.tail()

,Rk,Player,From,To,Draft,Tm,Lg,G,GS,Yrs,PB,AP1,AV
7865,7866,"(Chris Young, YounCh03)",2014,2014,,NYJ,NFL,0,,1,0,0,
7866,7867,"(Michael Zimmer, ZimmMi00)",2014,2014,,SEA,NFL,0,,1,0,0,
7867,7868,"(Ty Zimmerman, ZimmTy00)",2014,2014,,NOR,NFL,0,,1,0,0,
7868,7869,"(Michael Zordich, ZordMi01)",2014,2014,,NOR,NFL,0,,1,0,0,
7869,7870,"(Jordan Zumwalt, ZumwJo00)",2014,2014,6-192,PIT,NFL,0,,1,0,0,


In [19]:
av_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7870 entries, 0 to 7869
Data columns (total 13 columns):
Rk        7870 non-null object
Player    7870 non-null object
From      7870 non-null object
To        7870 non-null object
Draft     7870 non-null object
Tm        7870 non-null object
Lg        7870 non-null object
G         7870 non-null object
GS        7870 non-null object
Yrs       7870 non-null object
PB        7870 non-null object
AP1       7870 non-null object
AV        7870 non-null object
dtypes: object(13)
memory usage: 799.4+ KB


In [20]:
av_df[["Player", "Pfr_ID"]] = av_df.Player.apply(pd.Series)

In [21]:
av_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7870 entries, 0 to 7869
Data columns (total 14 columns):
Rk        7870 non-null object
Player    7870 non-null object
From      7870 non-null object
To        7870 non-null object
Draft     7870 non-null object
Tm        7870 non-null object
Lg        7870 non-null object
G         7870 non-null object
GS        7870 non-null object
Yrs       7870 non-null object
PB        7870 non-null object
AP1       7870 non-null object
AV        7870 non-null object
Pfr_ID    7870 non-null object
dtypes: object(14)
memory usage: 860.9+ KB


In [22]:
av_df.head()

,Rk,Player,From,To,Draft,Tm,Lg,G,GS,Yrs,PB,AP1,AV,Pfr_ID
0,1,Cam Newton,2011,2013,1-1,CAR,NFL,48,48,3,2,0,51,NewtCa00
1,2,Russell Wilson,2012,2014,3-75,SEA,NFL,48,48,3,2,0,51,WilsRu00
2,3,Patrick Willis,2007,2009,1-11,SFO,NFL,48,48,3,3,2,49,WillPa98
3,4,Patrick Peterson,2011,2013,1-5,CRD,NFL,48,48,3,3,2,48,PetePa00
4,5,Aaron Donald,2014,2016,1-13,RAM,NFL,48,44,3,3,2,46,DonaAa00


In [23]:
av_df.tail()

,Rk,Player,From,To,Draft,Tm,Lg,G,GS,Yrs,PB,AP1,AV,Pfr_ID
7865,7866,Chris Young,2014,2014,,NYJ,NFL,0,,1,0,0,,YounCh03
7866,7867,Michael Zimmer,2014,2014,,SEA,NFL,0,,1,0,0,,ZimmMi00
7867,7868,Ty Zimmerman,2014,2014,,NOR,NFL,0,,1,0,0,,ZimmTy00
7868,7869,Michael Zordich,2014,2014,,NOR,NFL,0,,1,0,0,,ZordMi01
7869,7870,Jordan Zumwalt,2014,2014,6-192,PIT,NFL,0,,1,0,0,,ZumwJo00


In [24]:
av_df.to_csv('data/first_3_season_av_since_2000_RAW_2018-03-13.csv', index=False)

# Scrape Player Info data

In [25]:
# use all letters cept X to pass into the url template and then scrape the data
abc = string.ascii_uppercase.replace("X", "")
player_info_dfs = []
player_info_errors = []
player_info_url_template = "http://www.pro-football-reference.com/players/{}/"
for i in abc:
    url = player_info_url_template.format(i)
    try:
        df = pfr.get_pfr_player_ids_and_info(url)
        player_info_dfs.append(df)
    except Exception as e:
        player_info_errors.append([url, e])

In [26]:
len(player_info_dfs)

25

In [27]:
len(player_info_errors)

0

In [28]:
player_info_df = pd.concat(player_info_dfs, ignore_index=True)

In [29]:
player_info_df.head()

,Player,Pfr_ID,Pos,From,To,Link,Text
0,Isaako Aaitui,AaitIs00,NT,2013,2014,/players/A/AaitIs00.htm,Isaako Aaitui (NT) 2013-2014
1,Joe Abbey,AbbeJo20,E,1948,1949,/players/A/AbbeJo20.htm,Joe Abbey (E) 1948-1949
2,Fay Abbott,AbboFa20,BB-FB-TB-QB-WB-E,1921,1929,/players/A/AbboFa20.htm,Fay Abbott (BB-FB-TB-QB-WB-E) 1921-1929
3,Vince Abbott,abbotvin01,K,1987,1988,/players/A/abbotvin01.htm,Vince Abbott (K) 1987-1988
4,Jared Abbrederis,AbbrJa00,WR,2014,2017,/players/A/AbbrJa00.htm,Jared Abbrederis (WR) 2014-2017


In [30]:
player_info_df.tail()

,Player,Pfr_ID,Pos,From,To,Link,Text
25372,Markus Zusevics,ZuseMa00,OL,2012,2012,/players/Z/ZuseMa00.htm,Markus Zusevics (OL) 2012-2012
25373,Jeremy Zuttah,ZuttJe20,G,2008,2016,/players/Z/ZuttJe20.htm,Jeremy Zuttah (G) 2008-2016
25374,Merle Zuver,ZuveMe20,G-C,1930,1930,/players/Z/ZuveMe20.htm,Merle Zuver (G-C) 1930-1930
25375,Tony Zuzzio,ZuzzTo20,G,1942,1942,/players/Z/ZuzzTo20.htm,Tony Zuzzio (G) 1942-1942
25376,Jim Zyntell,ZyntJi20,G,1933,1935,/players/Z/ZyntJi20.htm,Jim Zyntell (G) 1933-1935


In [31]:
player_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25377 entries, 0 to 25376
Data columns (total 7 columns):
Player    25377 non-null object
Pfr_ID    25377 non-null object
Pos       25377 non-null object
From      25377 non-null int64
To        25377 non-null int64
Link      25377 non-null object
Text      25377 non-null object
dtypes: int64(2), object(5)
memory usage: 1.4+ MB


In [32]:
player_info_df.to_csv('data/pfr_pids_and_info_RAW_2018-03-13.csv', index=False)